# Homework 2: Language Modeling
11-411/611 Natural Language Processing (Spring 2025)

- RELEASED: Tuesday, Feb 18th, 2025
- DUE: Thursday, March 13th 2025 11:59 pm EDT

Whether for transcribing spoken utterances as correct word sequences or generating coherent human-like text, language models are extremely useful.

In this assignment, you will be building your own language models powered by n-grams and RNNs.

### Submission Guidelines
**Programming:** 
- This notebook contains helpful test cases and additional information about the programming part of the HW. However, you are only required to submit `ngram_lm.py` and `rnn_lm.py` on Gradescope.
- We recommended that you first code in the notebook and then copy the corresponding methods/classes to `ngram_lm.py` and `rnn_lm.py`.

**Written:**
- Analysis questions would require you to run your code.
- You need to write your answers in a document and upload it alongside the programming components

### Upload (if using Colab) main.py and utils.py, and the data.zip file

In [1]:
# !unzip data.zip

import zipfile

with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

## Part 1: Language Models [60 points]

### Step 0: Preprocessing

In [2]:
!pip install transformers
!pip install requests
!pip install torch
!pip install tqdm

import math
import torch
import numpy as np
import torch.nn as nn
from collections import Counter
from torch.utils.data import DataLoader, Dataset

We provide you with a few functions in `utils.py` to read and preprocess your input data. Do not edit this file!

In [3]:
from utils import *

We have performed a round of preprocessing on the datasets.

- Each file contains one sentence per line.
- All punctuation marks have been removed.
- Each line is a sequences of tokens separated by whitespace.

#### Special Symbols ( Already defined in `utils.py` )
The start and end tokens will act as padding to the given sentences, to make sure they are correctly defined, print them here:

In [4]:
print("Sentence START symbol: {}".format(START))
print("Sentence END symbol: {}".format(EOS))
print("Unknown word symbol: {}".format(UNK))

Sentence START symbol: <s>
Sentence END symbol: </s>
Unknown word symbol: <UNK>


#### Reading and processing an example file

In [5]:
# Read the sample file
sample = read_file("data/sample.txt")
print(sample)

['We are never ever ever ever ever getting back together\n', 'We are the ones together we are back']


In [6]:
# Preprocess the content to add corresponding number of start and end tokens. Try out the method with n = 3 and n = 4 as well.
# Preprocessing example for bigrams (n=2)
sample = preprocess(sample, n=3)
for s in sample:
    print(s)

['<s>', '<s>', 'we', 'are', 'never', 'ever', 'ever', 'ever', 'ever', 'getting', 'back', 'together', '</s>']
['<s>', '<s>', 'we', 'are', 'the', 'ones', 'together', 'we', 'are', 'back', '</s>']


In [7]:
# Flattens a nested list into a 1D list.
flattened = flatten(sample)
print(flattened)

['<s>', '<s>', 'we', 'are', 'never', 'ever', 'ever', 'ever', 'ever', 'getting', 'back', 'together', '</s>', '<s>', '<s>', 'we', 'are', 'the', 'ones', 'together', 'we', 'are', 'back', '</s>']


### Step 1: N-Gram Language Model

#### TODO: Defining `get_ngrams()`

In [8]:
# #######################################
# # TODO: get_ngrams()
# #######################################
# def get_ngrams(list_of_words, n):
#     """
#     Returns a list of n-grams for a list of words.
#     Args
#     ----
#     list_of_words: List[str]
#         List of already preprocessed and flattened (1D) list of tokens e.g. ["<s>", "hello", "</s>", "<s>", "bye", "</s>"]
#     n: int
#         n-gram order e.g. 1, 2, 3
    
#     Returns:
#         n_grams: List[Tuple]
#             Returns a list containing n-gram tuples
#     """
#     raise NotImplementedError

# def get_ngrams(list_of_words, n):
#     """
#     Returns a list of n-grams for a list of words.
#     Args
#     ----
#     list_of_words: List[str]
#         List of already preprocessed and flattened (1D) list of tokens e.g. ["<s>", "hello", "</s>", "<s>", "bye", "</s>"]
#     n: int
#         n-gram order e.g. 1, 2, 3
    
#     Returns:
#         n_grams: List[Tuple]
#             Returns a list containing n-gram tuples
#     """
#     n_grams = []
#     for i in range(len(list_of_words) - n + 1):
#         n_gram = tuple(list_of_words[i:i+n])
#         n_grams.append(n_gram)
    
#     return n_grams

def get_ngrams(list_of_words, n):
    """
    Returns a list of n-grams for a list of words.
    
    Args:
    ----
    list_of_words: List[str]
        List of already preprocessed and flattened (1D) list of tokens.
    n: int
        n-gram order (e.g., 1 for unigrams, 2 for bigrams, etc.).
    
    Returns:
    -------
    List[Tuple]
        List of n-gram tuples.
    """
    return [tuple(list_of_words[i:i+n]) for i in range(len(list_of_words) - n + 1)]


In [9]:
#######################################
# TEST: get_ngrams()
#######################################
sample = preprocess(read_file("sample.txt"), n=3)
flattened = flatten(sample)

assert get_ngrams(flattened, 3) == [('<s>', '<s>', 'we'),
        ('<s>', 'we', 'are'),
        ('we', 'are', 'never'),
        ('are', 'never', 'ever'),
        ('never', 'ever', 'ever'),
        ('ever', 'ever', 'ever'),
        ('ever', 'ever', 'ever'),
        ('ever', 'ever', 'getting'),
        ('ever', 'getting', 'back'),
        ('getting', 'back', 'together'),
        ('back', 'together', '</s>'),
        ('together', '</s>', '<s>'),
        ('</s>', '<s>', '<s>'),
        ('<s>', '<s>', 'we'),
        ('<s>', 'we', 'are'),
        ('we', 'are', 'the'),
        ('are', 'the', 'ones'),
        ('the', 'ones', 'together'),
        ('ones', 'together', 'we'),
        ('together', 'we', 'are'),
        ('we', 'are', 'back'),
        ('are', 'back', '</s>')]

In [10]:
# Read the sample file
sample = read_file("sample.txt")
print("Raw sample data:", sample)

# Preprocess the content for bigrams (n=2)
sample_bigrams = preprocess(sample, n=2)
print("Preprocessed for bigrams:", sample_bigrams)

# Preprocess the content for trigrams (n=3)
sample_trigrams = preprocess(sample, n=3)
print("Preprocessed for trigrams:", sample_trigrams)

# Flatten the preprocessed data
flattened_bigrams = flatten(sample_bigrams)
print("Flattened bigrams:", flattened_bigrams)

flattened_trigrams = flatten(sample_trigrams)
print("Flattened trigrams:", flattened_trigrams)


# Extract bigrams and trigrams
bigrams = get_ngrams(flattened_bigrams, n=2)
trigrams = get_ngrams(flattened_trigrams, n=3)

print("Bigrams:", bigrams)
print("Trigrams:", trigrams)


from collections import Counter

# Count unique bigrams and trigrams
unique_bigrams = len(Counter(bigrams))
unique_trigrams = len(Counter(trigrams))

print("Unique bigrams:", unique_bigrams)
print("Unique trigrams:", unique_trigrams)

Raw sample data: ['We are never ever ever ever ever getting back together\n', 'We are the ones together we are back']
Preprocessed for bigrams: [['<s>', 'we', 'are', 'never', 'ever', 'ever', 'ever', 'ever', 'getting', 'back', 'together', '</s>'], ['<s>', 'we', 'are', 'the', 'ones', 'together', 'we', 'are', 'back', '</s>']]
Preprocessed for trigrams: [['<s>', '<s>', 'we', 'are', 'never', 'ever', 'ever', 'ever', 'ever', 'getting', 'back', 'together', '</s>'], ['<s>', '<s>', 'we', 'are', 'the', 'ones', 'together', 'we', 'are', 'back', '</s>']]
Flattened bigrams: ['<s>', 'we', 'are', 'never', 'ever', 'ever', 'ever', 'ever', 'getting', 'back', 'together', '</s>', '<s>', 'we', 'are', 'the', 'ones', 'together', 'we', 'are', 'back', '</s>']
Flattened trigrams: ['<s>', '<s>', 'we', 'are', 'never', 'ever', 'ever', 'ever', 'ever', 'getting', 'back', 'together', '</s>', '<s>', '<s>', 'we', 'are', 'the', 'ones', 'together', 'we', 'are', 'back', '</s>']
Bigrams: [('<s>', 'we'), ('we', 'are'), ('are'

#### **TODO:** Class `NGramLanguageModel()`

*Now*, we will define our LanguageModel class.

**Some Useful Variables:**
- self.model: `dict` of n-grams and their corresponding probabilities, keys being the tuple containing the n-gram, and the value being the probability of the n-gram.
- self.vocab: `dict` of unigram vocabulary with counts, keys being the words themselves and the values being their frequency.
- self.n: `int` value for n-gram order (e.g. 1, 2, 3).
- self.train_data: `List[List]` containing preprocessed **unflattened** train sentences. You will have to flatten it to use in the language model
- self.smoothing: `float` flag signifying the smoothing parameter.

In `lm.py`, we will be taking most of these argumemts from the command line using this command:

`python3 lm.py --train data/sample.txt --test data/sample.txt --n 3 --smoothing 0 --min_freq 1`

Note that we will not be using log probabilities in this section. Store the probabilities as they are, not in log space.

**Laplace Smoothing**

There are two ways to perform this:
- Either you calculate all possible n-grams at train time and calculate smooth probabilities for all of them, hence inflating the model (eager emoothing). You then use the probabilities as when required at test time. **OR**
- You calculate the probabilities for the **observed n-grams** at train time, using the smoothed likelihood formula, then if any unseen n-gram is observed at test time, you calculate the probability using the smoothed likelihood formula and store it in the model for future use (lazy smoothing).

You will be implementing lazy smoothing

**Perplexity**

Steps:
1. Flatten the test data.
2. Extract ngrams from the flattened data.
3. Calculate perplexity according to given formula. For unseen n-grams, calculate using smoothed likelihood and store the unseen n-gram probability in the labguage model `model` attribute:

$ppl(W_{test}) = ppl(W_1W_2 ... W_n)^{-1/n} $

Tips:
- Remember that product changes to summation under `log`. Take the log of probabilities, sum them up, and then exponentiate it to get back to the original scale.
- Make sure to `flatten()` your data before creating the n_grams using `get_ngrams()`.
- The test suite provided is **not exhaustive**.


In [11]:
#######################################
# TODO: NGramLanguageModel()
#######################################
class NGramLanguageModel():
    def __init__(self, n, train_data, alpha=1):
        """
        Language model class.

        Args:
        -----
        n: int
            n-gram order (e.g., 1, 2, 3).
        train_data: List[List]
            Preprocessed training data in sentence format.
        alpha: float
            Smoothing parameter.

        Attributes:
        -----
        self.tokens: Flattened list of tokens.
        self.vocab: Dictionary of word counts.
        self.model: Stores n-gram probabilities.
        self.n_grams_counts: Dictionary of n-gram counts.
        self.prefix_counts: Dictionary of (n-1)-gram counts.
        """
        self.n = n
        self.smoothing = alpha
        self.train_data = train_data

        # Preprocess data
        self.tokens = flatten(train_data)
        self.vocab = Counter(self.tokens)
        self.n_grams_counts = Counter()
        self.prefix_counts = Counter()
        self.model = {}

        # Construct model
        self.build()

    def build(self):
        """
        Computes n-gram and (n-1)-gram counts and stores initial probabilities.
        """
        ngram_list = get_ngrams(self.tokens, self.n)

        for ngram in ngram_list:
            self.n_grams_counts[ngram] += 1
            if self.n > 1:
                prefix = ngram[:-1]
                self.prefix_counts[prefix] += 1

        # Store precomputed probabilities
        for ngram in self.n_grams_counts.keys():
            self.model[ngram] = self.get_prob(ngram)

    def get_smooth_probabilities(self, ngrams):
        """
        Returns smoothed probabilities for a list of n-grams.

        Args:
        -----
        ngrams: list of tuples
            List of n-gram tuples.

        Returns:
        -----
        list of float
            List of probabilities.
        """
        return list(map(self.get_prob, ngrams))

    def get_prob(self, ngram):
        """
        Computes probability of an n-gram using Laplace smoothing.

        Args:
        -----
        ngram: tuple
            The n-gram tuple.

        Returns:
        -----
        float
            Probability of the n-gram.
        """
        vocab_size = len(self.vocab)
        count = self.n_grams_counts.get(ngram, 0)

        if self.n == 1:
            return (count + self.smoothing) / (sum(self.vocab.values()) + self.smoothing * vocab_size)

        prefix = ngram[:-1]
        prefix_count = self.prefix_counts.get(prefix, 0)
        return (count + self.smoothing) / (prefix_count + self.smoothing * vocab_size)

    def perplexity(self, test_data):
        """
        Computes perplexity using lazy Laplace smoothing.

        Args:
        -----
        test_data: List[List]
            Nested list of preprocessed test sentences.

        Returns:
        -----
        float
            Perplexity score.
        """
        test_tokens = flatten(test_data)
        test_ngrams = get_ngrams(test_tokens, self.n)

        if not test_ngrams:
            return float("inf")

        log_prob_total = sum(math.log(self.model.get(ngram, self.get_prob(ngram)) + 1e-10) for ngram in test_ngrams)
        return math.exp(-log_prob_total / len(test_ngrams))

# class NGramLanguageModel():
#     def __init__(self, n, train_data, alpha=1):
#         """
#         Language model class implementing lazy Laplace smoothing.
        
#         Args
#         ----
#         n: int
#             n-gram order
#         train_data: List[List]
#             Preprocessed unflattened list of sentences.
#         alpha: float
#             Smoothing parameter
        
#         Attributes
#         ----------
#         self.tokens: list
#             Flattened list of tokens in training corpus.
#         self.vocab: dict
#             Vocabulary dictionary with counts.
#         self.model: dict
#             Stores n-gram probabilities.
#         self.n_grams_counts: dict
#             Stores frequency of n-grams in training data.
#         self.prefix_counts: dict
#             Stores frequency of (n-1)-grams in training data.
#         """
#         self.n = n
#         self.train_data = train_data
#         self.smoothing = alpha
#         self.tokens = flatten(train_data)  # Flatten train data
#         self.vocab = Counter(self.tokens)  # Vocabulary with counts
#         self.model = {}  # Stores n-gram probabilities
#         self.n_grams_counts = Counter()
#         self.prefix_counts = Counter()
#         self.build()  # Build the model

#     def build(self):
#         """
#         Computes n-gram counts and prefix counts, then calculates probabilities for seen n-grams.
#         """
#         ngrams = get_ngrams(self.tokens, self.n)
#         for ngram in ngrams:
#             self.n_grams_counts[ngram] += 1
#             if self.n > 1:
#                 self.prefix_counts[ngram[:-1]] += 1
        
#         # Compute probabilities for observed n-grams (Lazy Smoothing applies at test time)
#         self.model = {ngram: self.get_prob(ngram) for ngram in self.n_grams_counts}

#     def get_smooth_probabilities(self, ngrams):
#         """
#         Returns a list of smoothed probabilities for a list of n-grams.

#         Args
#         ----
#         ngrams: list of tuples
#             List of n-grams for which probabilities are required.

#         Returns
#         -------
#         list
#             List of smoothed probabilities.
#         """
#         return [self.get_prob(ngram) for ngram in ngrams]

#     def get_prob(self, ngram):
#         """
#         Returns the probability of an n-gram using Laplace smoothing.

#         Args
#         ----
#         ngram: tuple
#             The n-gram tuple.

#         Returns
#         -------
#         float
#             The probability of the given n-gram.
#         """
#         vocab_size = len(self.vocab)
#         count = self.n_grams_counts.get(ngram, 0)

#         if self.n == 1:  # Unigram case
#             total_tokens = len(self.tokens)
#             return (count + self.smoothing) / (total_tokens + self.smoothing * vocab_size)

#         prefix = ngram[:-1]
#         prefix_count = self.prefix_counts.get(prefix, 0)
#         return (count + self.smoothing) / (prefix_count + self.smoothing * vocab_size)

#     def perplexity(self, test_data):
#         """
#         Computes perplexity for the given test data using lazy smoothing.

#         Args
#         ----
#         test_data: List[List]
#             Preprocessed nested list of sentences.

#         Returns
#         -------
#         float
#             Perplexity value.
#         """
#         test_tokens = flatten(test_data)
#         test_ngrams = get_ngrams(test_tokens, self.n)

#         prob_product = 1.0
#         total_ngrams = len(test_ngrams)

#         for ngram in test_ngrams:
#             if ngram not in self.model:  # Lazy smoothing: compute probability for unseen n-grams
#                 self.model[ngram] = self.get_prob(ngram)
#             prob_product *= self.model[ngram]  # Multiply raw probabilities

#         # Compute perplexity correctly
#         if total_ngrams == 0:
#             return float("inf")  # Edge case: No n-grams found
        
#         perplexity = pow(prob_product, -1 / total_ngrams)  # Using correct formula
#         return perplexity


# # class NGramLanguageModel():
# #     def __init__(self, n, train_data, alpha=1):
# #         self.n = n
# #         self.train_data = train_data
# #         self.smoothing = alpha
# #         self.tokens = flatten(train_data)  # Flatten train data
# #         self.vocab = Counter(self.tokens)  # Vocabulary with counts
# #         self.model = {}  # Stores n-gram probabilities
# #         self.n_grams_counts = Counter()
# #         self.prefix_counts = Counter()
# #         self.build()  # Build the model

# #     def build(self):
# #         """Precomputes probabilities for observed n-grams only."""
# #         ngrams = get_ngrams(self.tokens, self.n)
# #         for ngram in ngrams:
# #             self.n_grams_counts[ngram] += 1
# #             if self.n > 1:
# #                 self.prefix_counts[ngram[:-1]] += 1
# #         # Compute probabilities for **seen** n-grams (Lazy Smoothing happens at test time)
# #         self.model = {ngram: self.get_prob(ngram) for ngram in self.n_grams_counts}

# #     def get_smooth_probabilities(self, ngrams):
# #         """Returns a list of probabilities for a list of n-grams."""
# #         return [self.get_prob(ngram) for ngram in ngrams]

# #     def get_prob(self, ngram):
# #         """Returns the probability of an n-gram, applying Laplace smoothing."""
# #         vocab_size = len(self.vocab)
# #         count = self.n_grams_counts.get(ngram, 0)
        
# #         if self.n == 1:  # Unigram model
# #             total_tokens = len(self.tokens)
# #             return (count + self.smoothing) / (total_tokens + self.smoothing * vocab_size)
        
# #         prefix = ngram[:-1]
# #         prefix_count = self.prefix_counts.get(prefix, 0)
# #         return (count + self.smoothing) / (prefix_count + self.smoothing * vocab_size)

# #     def perplexity(self, test_data):
# #         """Computes perplexity using the correct formula & lazy smoothing."""
# #         test_tokens = flatten(test_data)
# #         test_ngrams = get_ngrams(test_tokens, self.n)
        
# #         log_prob_sum = 0.0
# #         for ngram in test_ngrams:
# #             if ngram not in self.model:  # Apply Lazy Smoothing for unseen n-grams
# #                 self.model[ngram] = self.get_prob(ngram)
# #             log_prob_sum += math.log(self.model[ngram] + 1e-10)  # Avoid log(0)

# #         N = len(test_ngrams)
# #         return math.exp(-log_prob_sum / max(1, N)) if N > 0 else float('inf')




In [12]:
#######################################
# TEST: NGramLanguageModel()
#######################################
# For the sake of understanding we will pass alpha as 0 (no smoothing), so that you gain intuition about the probabilities
sample = preprocess(read_file("data/sample.txt"), n=2)
test_lm = NGramLanguageModel(n=2, train_data=sample, alpha=0)

expected_vocab = Counter({'<s>': 2,
        'we': 3,
        'are': 3,
        'never': 1,
        'ever': 4,
        'getting': 1,
        'back': 2,
        'together': 2,
        '</s>': 2,
        'the': 1,
        'ones': 1})

expected_model = {('<s>', 'we'): 1.0,
        ('we', 'are'): 1.0,
        ('are', 'never'): 0.3333333333333333,
        ('never', 'ever'): 1.0,
        ('ever', 'ever'): 0.75,
        ('ever', 'getting'): 0.25,
        ('getting', 'back'): 1.0,
        ('back', 'together'): 0.5,
        ('together', '</s>'): 0.5,
        ('</s>', '<s>'): 1.0,
        ('are', 'the'): 0.3333333333333333,
        ('the', 'ones'): 1.0,
        ('ones', 'together'): 1.0,
        ('together', 'we'): 0.5,
        ('are', 'back'): 0.3333333333333333,
        ('back', '</s>'): 0.5}

assert test_lm.vocab == expected_vocab, f"Vocabulary mismatch! Expected: {expected_vocab}, but got: {test_lm.vocab}"

assert test_lm.model == expected_model, (
    f"Model mismatch! \n"
    f"Expected keys but missing: {set(expected_model.keys()) - set(test_lm.model.keys())}\n"
    f"Unexpected keys in model: {set(test_lm.model.keys()) - set(expected_model.keys())}\n"
    f"Discrepancies in probabilities: "
    f"{ {k: (expected_model[k], test_lm.model[k]) for k in expected_model if k in test_lm.model and expected_model[k] != test_lm.model[k]} }"
)

In [13]:
#######################################
# TEST smoothing: NGramLanguageModel()
#######################################
sample = preprocess(read_file("data/sample.txt"), n=2)
test_lm = NGramLanguageModel(n=2, train_data=sample, alpha=1)

expected_vocab_smoothing = Counter({'<s>': 2,
        'we': 3,
        'are': 3,
        'never': 1,
        'ever': 4,
        'getting': 1,
        'back': 2,
        'together': 2,
        '</s>': 2,
        'the': 1,
        'ones': 1})

expected_model_smoothing ={('<s>', 'we'): 0.23076923076923078,
        ('we', 'are'): 0.2857142857142857,
        ('are', 'never'): 0.14285714285714285,
        ('never', 'ever'): 0.16666666666666666,
        ('ever', 'ever'): 0.26666666666666666,
        ('ever', 'getting'): 0.13333333333333333,
        ('getting', 'back'): 0.16666666666666666,
        ('back', 'together'): 0.15384615384615385,
        ('together', '</s>'): 0.15384615384615385,
        ('</s>', '<s>'): 0.16666666666666666,
        ('are', 'the'): 0.14285714285714285,
        ('the', 'ones'): 0.16666666666666666,
        ('ones', 'together'): 0.16666666666666666,
        ('together', 'we'): 0.15384615384615385,
        ('are', 'back'): 0.14285714285714285,
        ('back', '</s>'): 0.15384615384615385}


assert test_lm.vocab == expected_vocab_smoothing, f"Vocabulary mismatch! Expected: {expected_vocab}, but got: {test_lm.vocab}"

assert test_lm.model == expected_model_smoothing, (
    f"Model mismatch! \n"
    f"Expected keys but missing: {set(expected_model_smoothing.keys()) - set(test_lm.model.keys())}\n"
    f"Unexpected keys in model: {set(test_lm.model.keys()) - set(expected_model_smoothing.keys())}\n"
    f"Discrepancies in probabilities: "
    f"{ {k: (expected_model_smoothing[k], test_lm.model[k]) for k in expected_model_smoothing if k in test_lm.model and expected_model_smoothing[k] != test_lm.model[k]} }"
)

In [14]:
#######################################
# TEST unigram: NGramLanguageModel()
#######################################
sample = preprocess(read_file("data/sample.txt"), n=1)
test_lm = NGramLanguageModel(n=1, train_data=sample, alpha=1)

expected_vocab_unigram = Counter({'<s>': 2,
        'we': 3,
        'are': 3,
        'never': 1,
        'ever': 4,
        'getting': 1,
        'back': 2,
        'together': 2,
        '</s>': 2,
        'the': 1,
        'ones': 1})

expected_model_unigram = {('<s>',): 0.09090909090909091,
        ('we',): 0.12121212121212122,
        ('are',): 0.12121212121212122,
        ('never',): 0.06060606060606061,
        ('ever',): 0.15151515151515152,
        ('getting',): 0.06060606060606061,
        ('back',): 0.09090909090909091,
        ('together',): 0.09090909090909091,
        ('</s>',): 0.09090909090909091,
        ('the',): 0.06060606060606061,
        ('ones',): 0.06060606060606061}


assert test_lm.vocab == expected_vocab_unigram, f"Vocabulary mismatch! Expected: {expected_vocab}, but got: {test_lm.vocab}"

assert test_lm.model == expected_model_unigram, (
    f"Model mismatch! \n"
    f"Expected keys but missing: {set(expected_model_unigram.keys()) - set(test_lm.model.keys())}\n"
    f"Unexpected keys in model: {set(test_lm.model.keys()) - set(expected_model_unigram.keys())}\n"
    f"Discrepancies in probabilities: "
    f"{ {k: (expected_model_unigram[k], test_lm.model[k]) for k in expected_model_unigram if k in test_lm.model and expected_model_unigram[k] != test_lm.model[k]} }"
)

In [15]:
#######################################
# TEST: perplexity()
#######################################
test_lm = NGramLanguageModel(n=2, train_data=sample, alpha=0)
test_ppl = test_lm.perplexity(sample)
assert test_ppl < 1.7
assert test_ppl > 0

test_lm = NGramLanguageModel(n=2, train_data=sample, alpha=1)
test_ppl = test_lm.perplexity(sample)
assert test_ppl < 5.0
assert test_ppl > 0

AssertionError: 

### Step 2: RNN Language Model
Recurrent Neural Networks (RNNs) are a class of neural networks designed to handle sequential data. Unlike traditional neural networks, which assume independence among inputs, RNNs utilize their internal state (memory) to process sequences of inputs. This makes them particularly well-suited for tasks where context and order matter.

Before diving into building RNN Language Models using PyTorch, it's essential to have a solid foundation in the following areas:
. We assume you have had a basic understanding of PyTorch and its core concepts, including tensors, autograd, modules (nn.Module), and how to construct simple neural networks using PyTorch. For more comprehensive learning, refer to the [PyTorch official tutorials](https://pytorch.org/tutorials/) and documentation.

#### Preparing the Data
The following Python code is used for loading and processing [GloVe (Global Vectors for Word Representation) embeddings](https://nlp.stanford.edu/projects/glove/). GloVe is an unsupervised learning algorithm for obtaining vector representations for words. These embeddings can be used in various natural language processing and machine learning tasks. You can download the 50d embeddings for this assignment from [Canvas](https://canvas.cmu.edu/courses/39596/files/10855662?module_item_id=5748476).

The `load_glove_embeddings(path)` function is used to load the GloVe embeddings from a file. The function takes a file path as an argument, reads the file line by line, and for each line, it splits the line into words and their corresponding embeddings, and stores them in a dictionary. The dictionary, embeddings_dict, maps words to their corresponding vector representations.

The `create_embedding_matrix(word_to_ix, embeddings_dict, embedding_dim)` function is used to create an embedding matrix from the loaded GloVe embeddings. This function takes a dictionary mapping words to their indices (`word_to_ix`), the dictionary of GloVe embeddings (`embeddings_dict`), and the dimension of the embeddings (`embedding_dim`) as arguments. It creates a zero matrix of size (vocab_size, embedding_dim) and then for each word in  `word_to_ix`, it checks if the word is in `embeddings_dict`. If it is, it assigns the corresponding GloVe vector to the word's index in the embedding matrix. If the word is not in the embeddings_dict, it assigns a random vector to the word's index in the embedding matrix.

The `glove_path` variable is the path to the GloVe file, and `glove_embeddings` is the dictionary of GloVe embeddings loaded using the `load_glove_embeddings` function. The `embedding_dim` variable is the dimension of the embeddings, and `embedding_matrix` is the embedding matrix created using the create_embedding_matrix function.

In [16]:
# Load the data
vocab, word_to_ix, ix_to_word, dataloader = loadfile("sample.txt")

In [17]:
def load_glove_embeddings(path):
    embeddings_dict = {}
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = torch.tensor([float(val) for val in values[1:]], dtype=torch.float)
            embeddings_dict[word] = vector
    return embeddings_dict

# Path to the GloVe file
glove_path = 'glove.6B.50d.txt'  # Update this path
glove_embeddings = load_glove_embeddings(glove_path)

def create_embedding_matrix(word_to_ix, embeddings_dict, embedding_dim):
    vocab_size = len(word_to_ix)
    embedding_matrix = torch.zeros((vocab_size, embedding_dim))
    for word, ix in word_to_ix.items():
        if word in embeddings_dict:
            embedding_matrix[ix] = embeddings_dict[word]
        else:
            embedding_matrix[ix] = torch.rand(embedding_dim)  # Random initialization for words not in GloVe
    return embedding_matrix

# Create the embedding matrix
embedding_dim = 50
embedding_matrix = create_embedding_matrix(word_to_ix, glove_embeddings, embedding_dim)

#### TODO: Defining the RNN Model

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class RNNLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, embedding_matrix):
        """
        RNN model class.
        
        Args
        ____
        vocab_size: int
            Size of the vocabulary
        embedding_dim: int
            Dimension of the word embeddings
        hidden_dim: int
            Dimension of the hidden state of the RNN
        embedding_matrix: torch.Tensor
            Pre-trained GloVe embeddings
        """
        super(RNNLanguageModel, self).__init__()
        
        self.device = torch.device("mps" if torch.backends.mps.is_available() 
                                 else "cuda" if torch.cuda.is_available() 
                                 else "cpu")
        print(f"Using device: {self.device}")

        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim

        # Embedding layer initialized with GloVe embeddings
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight = nn.Parameter(embedding_matrix)
        self.embedding.weight.requires_grad = False  # Freeze the embeddings

        # RNN layer
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, vocab_size)

        self.to(self.device)

    def forward(self, x, hidden=None):
        """
        The forward pass of the RNN model.
        
        Args
        ____
        x: torch.Tensor
            Input tensor of shape (batch_size, sequence_length)
        hidden: torch.Tensor
            Hidden state tensor of shape (num_layers, batch_size, hidden_dim)
        
        Returns
        -------
        out: torch.Tensor
            Output tensor of shape (batch_size, sequence_length, vocab_size)
        hidden: torch.Tensor
            Hidden state tensor of shape (num_layers, batch_size, hidden_dim)
        """
        x = x.to(self.device)
        if hidden is None:
            # Initialize hidden state if not provided
            hidden = torch.zeros(1, x.size(0), self.hidden_dim).to(self.device)
        else:
            hidden = hidden.to(self.device)

        # Embedding lookup
        x = self.embedding(x)

        # RNN forward pass
        out, hidden = self.rnn(x, hidden)

        # Fully connected layer
        out = self.fc(out)

        return out, hidden

    def generate_sentence(self, sequence, word_to_ix, ix_to_word, num_words, mode='max'):
        """
        Predicts the next words given a sequence.
        
        Args
        ____
        sequence: str
            Input sequence
        word_to_ix: dict
            Dictionary mapping words to their corresponding indices
        ix_to_word: dict
            Dictionary mapping indices to their corresponding words
        num_words: int
            Maximum number of words to predict
        mode: str
            Mode of prediction. 'max' or 'multinomial'
            'max' mode selects the word with maximum probability
            'multinomial' mode samples the word from the probability distribution
        
        Returns
        -------
        predicted_sequence: List[str]
            List of predicted words (excluding the initial sequence)
        """
        self.eval()
        predicted_sequence = []

        # Convert the input sequence to indices, handling unknown words
        sequence_indices = [word_to_ix.get(word, word_to_ix['<UNK>']) for word in sequence.split()]
        sequence_tensor = torch.tensor(sequence_indices, dtype=torch.long).unsqueeze(0).to(self.device)

        hidden = None
        for _ in range(num_words):
            with torch.no_grad():
                # Forward pass
                output, hidden = self.forward(sequence_tensor, hidden)
                output = output[:, -1, :]  # Get the last output

                # Predict the next word
                if mode == 'max':
                    _, next_word_idx = torch.max(output, dim=1)
                elif mode == 'multinomial':
                    probs = F.softmax(output, dim=1)
                    next_word_idx = torch.multinomial(probs, num_samples=1).squeeze()

                # Get the predicted word
                next_word = ix_to_word.get(next_word_idx.item(), '<UNK>')
                predicted_sequence.append(next_word)

                # Update the sequence tensor with the predicted word
                sequence_tensor = torch.tensor([[next_word_idx.item()]], dtype=torch.long).to(self.device)

        return predicted_sequence

#### Training the Model
The following code snippet provided is responsible for training the RNN language model. 

In [19]:
#######################################
# TEST: RNNLanguageModel() and training
#######################################
torch.manual_seed(11411)
# Hyperparameters
vocab_size = len(vocab)
embedding_dim = 50
hidden_dim = 32
num_epochs = 10

# Initialize the model, loss function, and optimizer
RNN = RNNLanguageModel(vocab_size, embedding_dim, hidden_dim, embedding_matrix)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(RNN.parameters(), lr=0.005)

lines = ""
# Training loop
for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs = inputs.to(RNN.device)
        targets = targets.to(RNN.device)
        
        RNN.zero_grad()
        output, _ = RNN(inputs)
        loss = criterion(output.view(-1, vocab_size), targets.view(-1))
        loss.backward()
        optimizer.step()
    
    line = f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Perplexity: {np.exp(loss.item())}'
    lines += line + "\n"
    print(line)

Using device: cpu
Epoch 1/10, Loss: 2.489769220352173, Perplexity: 12.058492944551825
Epoch 2/10, Loss: 2.3438196182250977, Perplexity: 10.42096474805513
Epoch 3/10, Loss: 2.212952136993408, Perplexity: 9.142666997991427
Epoch 4/10, Loss: 2.0943455696105957, Perplexity: 8.12012517431332
Epoch 5/10, Loss: 1.985386848449707, Perplexity: 7.281863818381051
Epoch 6/10, Loss: 1.883654236793518, Perplexity: 6.577496730070404
Epoch 7/10, Loss: 1.7871267795562744, Perplexity: 5.972268148099225
Epoch 8/10, Loss: 1.694391131401062, Perplexity: 5.443330682903116
Epoch 9/10, Loss: 1.6047165393829346, Perplexity: 4.976448775551447
Epoch 10/10, Loss: 1.5180013179779053, Perplexity: 4.5630958971646205


## Part 2: Written [40 points]. We have given some code for some of the written parts to make it easier for you.

In [20]:
# Load and preprocess the datasets
business_data = preprocess(read_file("business.txt"), n=2)
sports_data = preprocess(read_file("sport.txt"), n=2)

# Train the models
business_bigram_model = NGramLanguageModel(n=2, train_data=business_data, alpha=1)
business_trigram_model = NGramLanguageModel(n=3, train_data=business_data, alpha=1)

sports_bigram_model = NGramLanguageModel(n=2, train_data=sports_data, alpha=1)
sports_trigram_model = NGramLanguageModel(n=3, train_data=sports_data, alpha=1)

In [21]:
# Count unique n-grams
unique_business_bigrams = len(business_bigram_model.n_grams_counts)
unique_business_trigrams = len(business_trigram_model.n_grams_counts)

unique_sports_bigrams = len(sports_bigram_model.n_grams_counts)
unique_sports_trigrams = len(sports_trigram_model.n_grams_counts)

In [22]:
print("for unique business bigrams: " + str(unique_business_bigrams))
print("for unique business trigrams: " + str(unique_business_trigrams))
print("for unique sports bigrams: " + str(unique_sports_bigrams))
print("for unique sports trigrams: " + str(unique_sports_trigrams))

for unique business bigrams: 83819
for unique business trigrams: 141220
for unique sports bigrams: 77398
for unique sports trigrams: 135644


In [23]:
# Vocabulary size for business dataset
vocab_size_business = len(business_bigram_model.vocab)

# Vocabulary size for sports dataset
vocab_size_sports = len(sports_bigram_model.vocab)

In [24]:
# Possible n-grams for business dataset
possible_bigrams_business = vocab_size_business ** 2
possible_trigrams_business = vocab_size_business ** 3

# Possible n-grams for sports dataset
possible_bigrams_sports = vocab_size_sports ** 2
possible_trigrams_sports = vocab_size_sports ** 3

In [25]:
# Comparison for business dataset
print("Business Dataset:")
print(f"Unique bigrams: {unique_business_bigrams} (out of {possible_bigrams_business} possible bigrams)")
print(f"Unique trigrams: {unique_business_trigrams} (out of {possible_trigrams_business} possible trigrams)")

# Comparison for sports dataset
print("\nSports Dataset:")
print(f"Unique bigrams: {unique_sports_bigrams} (out of {possible_bigrams_sports} possible bigrams)")
print(f"Unique trigrams: {unique_sports_trigrams} (out of {possible_trigrams_sports} possible trigrams)")

Business Dataset:
Unique bigrams: 83819 (out of 141991056 possible bigrams)
Unique trigrams: 141220 (out of 1691965423296 possible trigrams)

Sports Dataset:
Unique bigrams: 77398 (out of 112508449 possible bigrams)
Unique trigrams: 135644 (out of 1193377118543 possible trigrams)


### **Written 4.2** – Song Attribution [8 points]

The cell below contains helper code to create an n-gram model for song attribution. You can use this code to train a model on the file provided in the train variable and compute the model's perplexity on the test lyrics. 

In [26]:
# Example code for Taylor Swift N-Gram LM
n = 3
smoothing = 0.1
min_freq = 1

train = read_file("taylor_swift.txt")
test = read_file("test_lyrics.txt")

train = preprocess(train, n)
test = preprocess(test, n)
lm = NGramLanguageModel(n, train, smoothing)

ppl = lm.perplexity(test)
print(ppl)

138.00661898624173


## Task 1: Compute Perplexity Scores for Tri-Gram Models

In [27]:
# Load and preprocess the unattributed song
test_lyrics = preprocess(read_file("test_lyrics.txt"), n=3)

# Train tri-gram models for each artist
taylor_swift_train = preprocess(read_file("taylor_swift.txt"), n=3)
green_day_train = preprocess(read_file("green_day.txt"), n=3)
ed_sheeran_train = preprocess(read_file("ed_sheeran.txt"), n=3)

taylor_swift_lm = NGramLanguageModel(n=3, train_data=taylor_swift_train, alpha=0.1)
green_day_lm = NGramLanguageModel(n=3, train_data=green_day_train, alpha=0.1)
ed_sheeran_lm = NGramLanguageModel(n=3, train_data=ed_sheeran_train, alpha=0.1)

# Compute perplexity for each artist
taylor_swift_ppl = taylor_swift_lm.perplexity(test_lyrics)
green_day_ppl = green_day_lm.perplexity(test_lyrics)
ed_sheeran_ppl = ed_sheeran_lm.perplexity(test_lyrics)

print("Perplexity scores for tri-gram models:")
print(f"(a) Taylor Swift: {taylor_swift_ppl}")
print(f"(b) Green Day: {green_day_ppl}")
print(f"(c) Ed Sheeran: {ed_sheeran_ppl}")

# Identify the most likely lyricist
most_likely_artist = min(
    ["Taylor Swift", "Green Day", "Ed Sheeran"],
    key=lambda x: taylor_swift_ppl if x == "Taylor Swift" else green_day_ppl if x == "Green Day" else ed_sheeran_ppl
)
print(f"Most likely lyricist (tri-gram): {most_likely_artist}")

Perplexity scores for tri-gram models:
(a) Taylor Swift: 138.00661898624173
(b) Green Day: 522.5399979005886
(c) Ed Sheeran: 521.2573608668955
Most likely lyricist (tri-gram): Taylor Swift


In [28]:
# Load and preprocess the unattributed song (Tri-Gram)
test_lyrics = preprocess(read_file("test_lyrics.txt"), n=3)

# Train tri-gram models for each artist
taylor_swift_train = preprocess(read_file("taylor_swift.txt"), n=3)
green_day_train = preprocess(read_file("green_day.txt"), n=3)
ed_sheeran_train = preprocess(read_file("ed_sheeran.txt"), n=3)

taylor_swift_lm = NGramLanguageModel(n=3, train_data=taylor_swift_train, alpha=0.1)
green_day_lm = NGramLanguageModel(n=3, train_data=green_day_train, alpha=0.1)
ed_sheeran_lm = NGramLanguageModel(n=3, train_data=ed_sheeran_train, alpha=0.1)

# Compute perplexity for each artist
taylor_swift_ppl = taylor_swift_lm.perplexity(test_lyrics)
green_day_ppl = green_day_lm.perplexity(test_lyrics)
ed_sheeran_ppl = ed_sheeran_lm.perplexity(test_lyrics)

print("1. Perplexity scores for tri-gram models:")
print(f"   (a) Taylor Swift: {taylor_swift_ppl}")
print(f"   (b) Green Day: {green_day_ppl}")
print(f"   (c) Ed Sheeran: {ed_sheeran_ppl}")


1. Perplexity scores for tri-gram models:
   (a) Taylor Swift: 138.00661898624173
   (b) Green Day: 522.5399979005886
   (c) Ed Sheeran: 521.2573608668955


In [29]:
# Identify the most likely lyricist based on lowest perplexity
most_likely_artist = min(
    ["Taylor Swift", "Green Day", "Ed Sheeran"],
    key=lambda x: taylor_swift_ppl if x == "Taylor Swift" else green_day_ppl if x == "Green Day" else ed_sheeran_ppl
)

print(f"\n2. Most likely lyricist (Tri-Gram): {most_likely_artist}")



2. Most likely lyricist (Tri-Gram): Taylor Swift


## Task 2: Task 3: Compute Perplexity Scores for Bi-Gram Models

In [30]:
# Load and preprocess the unattributed song (Bi-Gram)
test_lyrics_bi = preprocess(read_file("test_lyrics.txt"), n=2)

# Train bi-gram models for each artist
taylor_swift_train_bi = preprocess(read_file("taylor_swift.txt"), n=2)
green_day_train_bi = preprocess(read_file("green_day.txt"), n=2)
ed_sheeran_train_bi = preprocess(read_file("ed_sheeran.txt"), n=2)

# Create bi-gram language models
taylor_swift_lm_bi = NGramLanguageModel(n=2, train_data=taylor_swift_train_bi, alpha=0.1)
green_day_lm_bi = NGramLanguageModel(n=2, train_data=green_day_train_bi, alpha=0.1)
ed_sheeran_lm_bi = NGramLanguageModel(n=2, train_data=ed_sheeran_train_bi, alpha=0.1)

# Compute perplexity for each artist (Bi-Gram)
taylor_swift_ppl_bi = taylor_swift_lm_bi.perplexity(test_lyrics_bi)
green_day_ppl_bi = green_day_lm_bi.perplexity(test_lyrics_bi)
ed_sheeran_ppl_bi = ed_sheeran_lm_bi.perplexity(test_lyrics_bi)

print("\n3. Perplexity scores for bi-gram models:")
print(f"   (a) Taylor Swift: {taylor_swift_ppl_bi}")
print(f"   (b) Green Day: {green_day_ppl_bi}")
print(f"   (c) Ed Sheeran: {ed_sheeran_ppl_bi}")



3. Perplexity scores for bi-gram models:
   (a) Taylor Swift: 90.36503789935158
   (b) Green Day: 286.38891953790426
   (c) Ed Sheeran: 298.3478048232207


In [31]:
# Identify the most likely lyricist using Bi-Gram
most_likely_artist_bi = min(
    ["Taylor Swift", "Green Day", "Ed Sheeran"],
    key=lambda x: taylor_swift_ppl_bi if x == "Taylor Swift" else green_day_ppl_bi if x == "Green Day" else ed_sheeran_ppl_bi
)

print(f"\n4. Most likely lyricist (Bi-Gram): {most_likely_artist_bi}")



4. Most likely lyricist (Bi-Gram): Taylor Swift


## Task 5: Compute Accuracy Scores for Bi-Gram & Tri-Gram

In [34]:
import os
from pathlib import Path

# 1️⃣ Set Parameters for the Model
n = 3  # Change to 2 for Bi-Gram, 3 for Tri-Gram
smoothing = 0.1

# 2️⃣ Define the Path to the Lyrics Folder
data_dir = Path(r"C:\Users\STUDENT\Downloads\handout (3)\handout\data\data\lyrics")

# 3️⃣ Split Datasets into Train & Test
train_splits, test_splits = split_and_save_datasets(data_dir=data_dir)

# 4️⃣ Get All Text Files (Ignoring "test_lyrics.txt")
files = [f for f in data_dir.glob("*.txt") if f.name != "test_lyrics.txt"]

# 5️⃣ Train and Evaluate Models
total = 0
correct = 0

for f in files:
    # Load Training Data for Each Artist
    train = train_splits[f.name]
    
    # Train the N-Gram Model
    lm = NGramLanguageModel(n, train, smoothing)
    
    # Compute Perplexity Against All Other Artists
    perplexities = {}
    for f_2 in files:
        test = test_splits[f_2.name]
        ppl = lm.perplexity(test)
        perplexities[f_2.name] = ppl
    
    # Identify the Most Likely Artist (Lowest Perplexity)
    min_perplexity_file = min(perplexities, key=perplexities.get)

    # 6️⃣ Check if the Model Correctly Identified the Artist
    total += 1
    if min_perplexity_file == f.name:
        correct += 1

    print(f"Min perplexity for model trained on {f.name}: {min_perplexity_file}")

# 7️⃣ Compute Overall Accuracy
accuracy = 100 * (correct / total)
print(f"\nAccuracy score for {n}-gram model: {accuracy:.2f}%")


Min perplexity for model trained on billie_eillish.txt: billie_eillish.txt
Min perplexity for model trained on ed_sheeran.txt: taylor_swift.txt
Min perplexity for model trained on green_day.txt: taylor_swift.txt
Min perplexity for model trained on taylor_swift.txt: taylor_swift.txt

Accuracy score for 3-gram model: 50.00%


Below, contains helper code for questions 4.2.4 and 4.2.5. We use a helper function split_and_save_datasets, to save the train and test data for each artist separately. Then, we train an n-gram model for each artists train set, and evaluate its perplexity on each artist's test set. By checking the artist with the lowest perplexity, we can compute the accuracy of each model. 

In [42]:
# Modify this for your particular bi-gram or tri-gram model
n = 3
smoothing = 0.1
min_freq = 2

# Run the splitting and saving process for the lyrics data
train_splits, test_splits = split_and_save_datasets(data_dir="data/lyrics/")
files = [f for f in Path("data/lyrics/").glob("*.txt") if f.name != "test_lyrics.txt"]

total = 0
correct = 0
for f in files:
  # Train a n-gram for each artists' lyrics
  train = train_splits[f.name]
  lm = NGramLanguageModel(n, train, smoothing)

  # Compute the model's perplexity on each other artist 
  perplexities = {}
  for f_2 in files:
    test = test_splits[f_2.name]
    ppl = lm.perplexity(test)
    perplexities[f_2.name] = ppl

  # Which artist has the lowest perplexity for our model
  min_perplexity_file = min(perplexities, key=perplexities.get)

  # Check whether the lowest perplexity artist is the same as which our model 
  # was trained on. 
  total += 1
  if min_perplexity_file == f.name:
    correct += 1
  
  print(f"Min perplexity for model trained on {f.name}: {min_perplexity_file}")


print(f"Accuracy score for {n}-gram model: ", 100 * (correct / total), "%")

Min perplexity for model trained on billie_eillish.txt: billie_eillish.txt
Min perplexity for model trained on ed_sheeran.txt: taylor_swift.txt
Min perplexity for model trained on green_day.txt: taylor_swift.txt
Min perplexity for model trained on taylor_swift.txt: taylor_swift.txt
Accuracy score for 3-gram model:  50.0 %


### **Written 4.3.1** –  Intro to Decoding [8 points]

Please take a look at and understand the functions: `best_candidate()`, `top_k_best_candidates()` and `generate_sentences_from_phrase()` in `utils.py`.

In [43]:
n = 3
smoothing = 0.1
min_freq = 1

In [44]:
train = read_file("data/lyrics/taylor_swift.txt")
train = preprocess(train, n)
lm = NGramLanguageModel(n, train, smoothing)

In [45]:
s1 = ("the", "tortured", "poets", "department")

s2 = ("so", "long", "london")

s3 = ("down", "bad")

In [46]:
print(top_k_best_candidates(lm, s1, 5, without=['<s>', '</s>']))

('</s>', 1)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from utils import loadfile, load_glove_embeddings, create_embedding_matrix

class RNNLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, embedding_matrix):
        super(RNNLanguageModel, self).__init__()
        
        self.device = torch.device("mps" if torch.backends.mps.is_available() 
                                 else "cuda" if torch.cuda.is_available() 
                                 else "cpu")
        print(f"Using device: {self.device}")

        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim

        # Embedding layer initialized with GloVe embeddings
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight = nn.Parameter(embedding_matrix)
        self.embedding.weight.requires_grad = False  # Freeze the embeddings

        # RNN layer
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, vocab_size)

        self.to(self.device)

    def forward(self, x, hidden=None):
        x = x.to(self.device)
        if hidden is None:
            hidden = torch.zeros(1, x.size(0), self.hidden_dim).to(self.device)
        else:
            hidden = hidden.to(self.device)

        # Embedding lookup
        x = self.embedding(x)

        # RNN forward pass
        out, hidden = self.rnn(x, hidden)

        # Fully connected layer
        out = self.fc(out)

        return out, hidden

    def generate_sentence(self, sequence, word_to_ix, ix_to_word, num_words, mode='max'):
        self.eval()
        predicted_sequence = []

        # Convert the input sequence to indices, handling unknown words
        sequence_indices = [word_to_ix.get(word, word_to_ix[UNK]) for word in sequence.split()]
        sequence_tensor = torch.tensor(sequence_indices, dtype=torch.long).unsqueeze(0).to(self.device)

        hidden = None
        for _ in range(num_words):
            with torch.no_grad():
                # Forward pass
                output, hidden = self.forward(sequence_tensor, hidden)
                output = output[:, -1, :]  # Get the last output

                # Predict the next word
                if mode == 'max':
                    _, next_word_idx = torch.max(output, dim=1)
                elif mode == 'multinomial':
                    probs = F.softmax(output, dim=1)
                    next_word_idx = torch.multinomial(probs, num_samples=1).squeeze()

                # Get the predicted word
                next_word = ix_to_word.get(next_word_idx.item(), UNK)
                predicted_sequence.append(next_word)

                # Update the sequence tensor with the predicted word
                sequence_tensor = torch.tensor([[next_word_idx.item()]], dtype=torch.long).to(self.device)

        return predicted_sequence

# Load the dataset
vocab, word_to_ix, ix_to_word, dataloader = loadfile("data/lyrics/taylor_swift.txt")

# Hyperparameters
vocab_size = len(vocab)
embedding_dim = 50
hidden_dim = 128
num_epochs = 10
learning_rate = 0.001

# Load GloVe embeddings
glove_path = "glove.6B.50d.txt"  # Update this path
glove_embeddings = load_glove_embeddings(glove_path)

# Create the embedding matrix
embedding_matrix = create_embedding_matrix(word_to_ix, glove_embeddings, embedding_dim)

# Initialize the model
model = RNNLanguageModel(vocab_size, embedding_dim, hidden_dim, embedding_matrix)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for inputs, targets in dataloader:
        inputs = inputs.to(model.device)
        targets = targets.to(model.device)

        # Forward pass
        hidden = None
        outputs, hidden = model(inputs, hidden)
        loss = criterion(outputs.view(-1, model.vocab_size), targets.view(-1))

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(dataloader)}")

# Predict top 5 words for each track title
s1 = ("the", "tortured", "poets", "department")
s2 = ("so", "long", "london")
s3 = ("down", "bad")

print("s1:", top_k_best_candidates(model, s1, k=5, without=[START, EOS]))
print("s2:", top_k_best_candidates(model, s2, k=5, without=[START, EOS]))
print("s3:", top_k_best_candidates(model, s3, k=5, without=[START, EOS]))

# Generate a sentence using 'max' mode
generated_sentence_max = model.generate_sentence("the tortured poets department", word_to_ix, ix_to_word, num_words=10, mode='max')
print("Generated sentence (max mode):", generated_sentence_max)

# Generate a sentence using 'multinomial' mode
generated_sentence_multinomial = model.generate_sentence("the tortured poets department", word_to_ix, ix_to_word, num_words=10, mode='multinomial')
print("Generated sentence (multinomial mode):", generated_sentence_multinomial)

Using device: cpu
Epoch 1/10, Loss: 2.5897959001072666
Epoch 2/10, Loss: 1.9786226121894002
Epoch 3/10, Loss: 1.8724897524848845
Epoch 4/10, Loss: 1.8223498441743418
Epoch 5/10, Loss: 1.7907547225442806
Epoch 6/10, Loss: 1.7688029116316264


### **Written 4.3.2** – Text Generation [8 points]

For this subtask, train an RNN LM using `data/taylor_swift.txt`

In this part, we will try the first two approaches to generate sentences.

Q1. Use `predict_next_words()` method to generate sentences after the provided phrases from `s1` to `s3`. Use modes `max` and `multinomial`. Report one of your favorite generations (for any strategy or phrase).

Q2. Which decoding strategy did you like better and why?

In [ ]:
s1 = "the tortured poets department"

s2 = "so long, london"

s3 = "down bad"

In [ ]:
# Calculate your RNN model's perplexity
torch.manual_seed(11411)

vocab, word_to_ix, ix_to_word, dataloader = loadfile("data/lyrics/taylor_swift.txt")

# Hyperparameters
vocab_size = len(vocab)
embedding_dim = 50
hidden_dim = 32
num_epochs = 10

glove_embeddings = load_glove_embeddings('glove.6B.50d.txt')
embedding_matrix = create_embedding_matrix(word_to_ix, glove_embeddings, embedding_dim)

# Initialize the model, loss function, and optimizer
RNN = RNNLanguageModel(vocab_size, embedding_dim, hidden_dim, embedding_matrix)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(RNN.parameters(), lr=0.005)

perplexity = 0
# Training loop
for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs = inputs.to(RNN.device)
        targets = targets.to(RNN.device)
        
        RNN.zero_grad()
        output, _ = RNN(inputs)
        loss = criterion(output.view(-1, vocab_size), targets.view(-1))
        loss.backward()
        optimizer.step()
    
    perplexity = np.exp(loss.item())

perplexity

In [ ]:
sentence = s1
predicted_words_sequence = RNN.generate_sentence(sentence, word_to_ix, ix_to_word, 10, mode='multinomial')
print(sentence + ' ' + ' '.join(predicted_words_sequence))

**Aside (for fun!)**: Train your LM on Taylor Swift lyrics and generate the next hit!

### **Written 4.4** – Battle of the LMs: GPT-2, Trigram and RNN [8 points]

For this subtask, you will be generating text and comparing GPT-2 with your n-gram and RNN language models. 

Generative pretrained transformer (GPT) is a neural language model series created by OpenAI. The n-gram language model you trained has on average around 10K-20K parameters (`len(lm.model)`.) Compare that to the 175 billion parameters of GPT-3, which is likely much smaller than more recent iterations (though they don't tell us anymore)!

Let's see how GPT-2 compares to the LMs you trained in Written 4.3.1 on the `data/bbc/tech-small.txt` dataset.

In [ ]:
# Calculate your n-gram model's perplexity
test = preprocess(read_file("data/bbc/tech-small.txt"), 3)
NGram = NGramLanguageModel(n=3, train_data=test)
NGram.perplexity(test)

In [ ]:
# Calculate your RNN model's perplexity
torch.manual_seed(11411)

vocab, word_to_ix, ix_to_word, dataloader = loadfile("data/bbc/tech-small.txt")

# Hyperparameters
vocab_size = len(vocab)
embedding_dim = 50
hidden_dim = 32
num_epochs = 10

glove_embeddings = load_glove_embeddings('glove.6B.50d.txt')
embedding_matrix = create_embedding_matrix(word_to_ix, glove_embeddings, embedding_dim)

# Initialize the model, loss function, and optimizer
RNN = RNNLanguageModel(vocab_size, embedding_dim, hidden_dim, embedding_matrix)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(RNN.parameters(), lr=0.005)

lines = ""
# Training loop
for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs = inputs.to(RNN.device)
        targets = targets.to(RNN.device)
        
        RNN.zero_grad()
        output, _ = RNN(inputs)
        loss = criterion(output.view(-1, vocab_size), targets.view(-1))
        loss.backward()
        optimizer.step()
    
    line = f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Perplexity: {np.exp(loss.item())}'
    lines += line + "\n"
    print(line)

#### Computing GPT-2's perplexity on the test set

You need to enable a GPU runtime from the Colab `Runtime` menu option (you can also use your computer if you have an accelerator). Go to `Runtime` → `Change Runtime Type` → `Hardware Accelerator (GPU)`

In [ ]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import torch

model_id = "distilgpt2"
model = GPT2LMHeadModel.from_pretrained(model_id)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

In [ ]:
test = read_file("data/bbc/tech-small.txt")
encodings = tokenizer("\n\n".join(test), return_tensors="pt")

In [ ]:
from tqdm import tqdm

max_length = model.config.n_positions
stride = 100

nlls = []
for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
    begin_loc = max(i + stride - max_length, 0)
    end_loc = min(i + stride, encodings.input_ids.size(1))
    trg_len = end_loc - i  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc]
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)
        neg_log_likelihood = outputs[0] * trg_len

    nlls.append(neg_log_likelihood)

ppl = torch.exp(torch.stack(nlls).sum() / end_loc)

In [ ]:
print("Perplexity using GPT2:", ppl.item())